In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset

In [2]:
# Load and preprocess
df = pd.read_csv('ICU_scores.csv')
df = df.drop(columns=["Unnamed: 0", "subject_id", "hadm_id", "icustay_id", "READMIT_30D_ICU"])
df = df.dropna()

X = df.drop(columns=["READMIT_30D"])
y = df["READMIT_30D"]

In [3]:
# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply SMOTE for oversampling
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_scaled, y)

In [4]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

In [5]:
# Convert to tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).unsqueeze(1)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

In [6]:
# Dataloader
train_ds = TensorDataset(X_train_tensor, y_train_tensor)
train_dl = DataLoader(train_ds, batch_size=64, shuffle=True)

In [7]:
# CNN model
class TabularCNN(nn.Module):
    def __init__(self, input_dim):
        super(TabularCNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=3, padding=1)
        self.pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = self.pool(x).squeeze(-1)
        x = self.fc(x)
        return self.sigmoid(x)

In [8]:
# Training
model = TabularCNN(input_dim=X_train.shape[1])
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(10):
    for xb, yb in train_dl:
        preds = model(xb).squeeze()
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

Epoch 1, Loss: 0.6709
Epoch 2, Loss: 0.5353
Epoch 3, Loss: 0.7330
Epoch 4, Loss: 0.5493
Epoch 5, Loss: 0.5628
Epoch 6, Loss: 0.5527
Epoch 7, Loss: 0.4831
Epoch 8, Loss: 0.5177
Epoch 9, Loss: 0.5213
Epoch 10, Loss: 0.7560


In [9]:
# Evaluation
with torch.no_grad():
    preds = model(X_test_tensor).squeeze()
    preds_class = (preds > 0.5).int()
    acc = (preds_class == y_test_tensor.int()).float().mean()
    print(f"Test Accuracy: {acc:.4f}")

Test Accuracy: 0.6908


In [11]:
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE

In [12]:
# Apply SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

In [13]:
# Create pipeline: Scaling + Linear SVM + Probabilities
base_svm = LinearSVC(C=1.0, max_iter=10000, random_state=42)
calibrated_svm = CalibratedClassifierCV(base_svm, method='sigmoid', cv=3)

# Train the calibrated SVM
pipeline = make_pipeline(StandardScaler(), calibrated_svm)
pipeline.fit(X_resampled, y_resampled)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('calibratedclassifiercv',
                 CalibratedClassifierCV(cv=3,
                                        estimator=LinearSVC(max_iter=10000,
                                                            random_state=42)))])

In [14]:
# Predict and evaluate
y_pred = pipeline.predict(X_test)
y_proba = pipeline.predict_proba(X_test)[:, 1]  # If you need probability scores

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.6697140381282496
Classification Report:
               precision    recall  f1-score   support

           0       0.66      0.70      0.68     11675
           1       0.68      0.64      0.66     11405

    accuracy                           0.67     23080
   macro avg       0.67      0.67      0.67     23080
weighted avg       0.67      0.67      0.67     23080

